In [1]:
!wget https://www.dropbox.com/scl/fi/avbnxsit6j3htk6la2u3l/stacksample.zip?rlkey=1ah8g9idp9t3scfxw83dp0nie&st=qzlmvxne&dl=0

--2025-05-29 16:00:51--  https://www.dropbox.com/scl/fi/avbnxsit6j3htk6la2u3l/stacksample.zip?rlkey=1ah8g9idp9t3scfxw83dp0nie
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4829a3b2994cf547cf0fe34626.dl.dropboxusercontent.com/cd/0/inline/CqmjFBig_7QP57t0BtW8oOtMJauK7qWDQVEnDiEEHe81HWNQva4gpi_Nra_cTz3jZxLOWN5TOQYRJWmqRzA6wc1Hcz2die4Eh_p4YlO1FtCiIfw0MxpoaXW7FAA6rBaBFDH37BrsQLm2bDNS2_2VL5DP/file# [following]
--2025-05-29 16:00:52--  https://uc4829a3b2994cf547cf0fe34626.dl.dropboxusercontent.com/cd/0/inline/CqmjFBig_7QP57t0BtW8oOtMJauK7qWDQVEnDiEEHe81HWNQva4gpi_Nra_cTz3jZxLOWN5TOQYRJWmqRzA6wc1Hcz2die4Eh_p4YlO1FtCiIfw0MxpoaXW7FAA6rBaBFDH37BrsQLm2bDNS2_2VL5DP/file
Resolving uc4829a3b2994cf547cf0fe34626.dl.dropboxusercontent.com (uc4829a3b2994cf547cf0fe34626.dl.dropboxusercontent.com)... 162.125.5.15, 262

In [2]:
! unzip stacksample.zip?rlkey=1ah8g9idp9t3scfxw83dp0nie

Archive:  stacksample.zip?rlkey=1ah8g9idp9t3scfxw83dp0nie
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                


In [3]:
import pandas as pd
import numpy as np


In [4]:
df=pd.read_csv('/content/Questions.csv', encoding='ISO-8859-1')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264216 entries, 0 to 1264215
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Id            1264216 non-null  int64  
 1   OwnerUserId   1249762 non-null  float64
 2   CreationDate  1264216 non-null  object 
 3   ClosedDate    55959 non-null    object 
 4   Score         1264216 non-null  int64  
 5   Title         1264216 non-null  object 
 6   Body          1264216 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 67.5+ MB


In [6]:
tags=pd.read_csv('/content/Tags.csv', encoding='ISO-8859-1')

In [7]:
tags.head(10)

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn
5,90,branch
6,90,branching-and-merging
7,120,sql
8,120,asp.net
9,120,sitemap


In [8]:
count_tag=pd.DataFrame(tags['Tag'].value_counts())

In [9]:
count_tag.reset_index(inplace=True)

In [10]:
popular_tags = count_tag['Tag'].head(20)

In [11]:
popular_tags

,Tag
0,javascript
1,java
2,c#
3,php
4,android
5,jquery
6,python
7,html
8,c++
9,ios


In [12]:
tag_used=popular_tags.unique()

In [13]:
tag_used

array(['javascript', 'java', 'c#', 'php', 'android', 'jquery', 'python',
       'html', 'c++', 'ios', 'mysql', 'css', 'sql', 'asp.net',
       'objective-c', 'ruby-on-rails', '.net', 'c', 'iphone', 'angularjs'],
      dtype=object)

In [14]:
merged_tag=pd.merge(tags,popular_tags, on='Tag')

In [15]:
merged_tag.head(20)

,Id,Tag
0,120,sql
1,120,asp.net
2,260,c#
3,260,.net
4,330,c++
5,470,.net
6,650,c#
7,930,c#
8,1010,c#
9,1010,asp.net


In [16]:
tags['Tag'] = tags['Tag'].astype(str)

In [17]:
final_tags = merged_tag.groupby('Id')['Tag'].apply(lambda x: ' '.join(sorted(x.unique()))).reset_index()

In [18]:
final_tags

,Id,Tag
0,120,asp.net sql
1,260,.net c#
2,330,c++
3,470,.net
4,650,c#
...,...,...
850983,40143150,javascript
850984,40143190,python
850985,40143210,php
850986,40143340,android


In [19]:
df.shape

(1264216, 7)

In [20]:
Final_tags= final_tags.drop_duplicates(subset='Id')

In [21]:
Final_tags.shape

(850988, 2)

In [22]:
df.drop(['OwnerUserId','CreationDate','ClosedDate'],axis=1, inplace=True)

#Filtering Questions.csv which contains only important tags

In [23]:
Que_Id=Final_tags['Id'].unique()

In [24]:
ques=df[df['Id'].isin(Que_Id)]

In [25]:
ques.shape

(850988, 4)

# Merging Popular tags with question

In [26]:
data_df= pd.merge(ques,Final_tags,on = 'Id',how='inner')

In [27]:
data_df.shape

(850988, 5)

In [28]:
data_df

,Id,Score,Title,Body,Tag
0,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,asp.net sql
1,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,.net c#
2,330,29,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,c++
3,470,13,Homegrown consumption of web services,<p>I've been writing a few web services for a ...,.net
4,650,79,Automatically update version number,<p>I would like the version property of my app...,c#
...,...,...,...,...,...
850983,40143150,0,"YouTube iFrame API: no ready call, no error call","<p>I'm trying to detect the ""flash out of date...",javascript
850984,40143190,1,How to execute multiline python code from a ba...,<p>I need to extend a shell script (bash). As ...,python
850985,40143210,0,URL routing in PHP (MVC),<p>I am building a custom MVC project and I ha...,php
850986,40143340,1,Obfuscating code in android studio,<p>Under minifyEnabled I changed from false to...,android


In [29]:
from bs4 import BeautifulSoup
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

from nltk.tokenize.toktok import ToktokTokenizer
import html

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [30]:
token = ToktokTokenizer()
lemma = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))
punct = r'!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'


def preprocess(text):
    soup = BeautifulSoup(text, "html.parser")
    for code in soup.find_all(['code', 'pre']):
        code.decompose()
    text = soup.get_text()

    # Unescape HTML entities (e.g. &lt; becomes <)
    text = html.unescape(text)

    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)

    # Lowercase and strip
    text = text.lower().strip()

    # Remove punctuation and tokenize
    regex = re.compile('[%s]' % re.escape(punct))
    tokens = token.tokenize(text)
    processed = []
    for w in tokens:
        if w in tag_used:
            processed.append(w)
        else:
            w = regex.sub('', w)
            if w and w not in stop_words:
                lemma_word = lemma.lemmatize(w, pos='v')
                processed.append(lemma_word)

    return ' '.join(processed)

In [31]:
import warnings
warnings.filterwarnings('ignore')

In [64]:
less_data=data_df.head(10000)

In [65]:
less_data['text']=less_data['Title']+less_data['Body']

In [66]:
less_data['body_new']=less_data['text'].apply(preprocess)

In [67]:
#less_data['Title_new']=less_data['Title'].apply(preprocess)

In [68]:
less_data['body_new'][0]

'asp.net site mapshas anyone get experience create sql-based asp.net site-map providers get default xml file work properly menu sitemappath control need way users site create modify page dynamically need tie page view permissions standard membership system well'

In [69]:
less_data.head()

,Id,Score,Title,Body,Tag,text,body_new
0,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,asp.net sql,ASP.NET Site Maps<p>Has anyone got experience ...,asp.net site mapshas anyone get experience cre...
1,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,.net c#,Adding scripting functionality to .NET applica...,add script functionality .net applicationsi li...
2,330,29,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,c++,Should I use nested classes in this case?<p>I ...,use nest class casei work collection class use...
3,470,13,Homegrown consumption of web services,<p>I've been writing a few web services for a ...,.net,Homegrown consumption of web services<p>I've b...,homegrown consumption web servicesi write web ...
4,650,79,Automatically update version number,<p>I would like the version property of my app...,c#,Automatically update version number<p>I would ...,automatically update version numberi would lik...


In [70]:
 x=less_data['body_new']
y=less_data['Tag']

In [71]:
y_final = [tag_string.split() for tag_string in y]

In [72]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical

In [73]:
from sklearn.preprocessing import MultiLabelBinarizer

In [74]:
mlb=MultiLabelBinarizer()

In [75]:
y=mlb.fit_transform(y_final)

In [76]:
len(mlb.classes_)

19

In [77]:
X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size=0.15)

In [78]:
Y_train.shape

(8500, 19)

In [79]:
Y_test.shape

(1500, 19)

In [80]:
sent_lens=[]
for sent in less_data['body_new']:
    sent_lens.append(len(word_tokenize(sent)))

In [81]:
max(sent_lens)

669

In [82]:
np.quantile(sent_lens,0.95)

np.float64(126.0)

In [83]:
max_len = 126

tok = Tokenizer(char_level=False,split=' ')

tok.fit_on_texts(X_train)

In [84]:
sequences_train = tok.texts_to_sequences(X_train)

In [85]:
vocab_len=len(tok.index_word.keys())

In [86]:
vocab_len

28197

In [87]:
sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len,padding='post')

In [88]:
sequences_matrix_train.shape

(8500, 126)

In [100]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])

    layer = Embedding(vocab_len+1,100,input_length=max_len,
                      mask_zero=True)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(len(mlb.classes_),name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [101]:
model = RNN()
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer) │ (None, 126)       │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 126, 100)  │  2,819,800 │ inputs[0][0]      │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_2         │ (None, 126)       │          0 │ inputs[0][0]      │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 64)        │     42,240 │ embedding_2[0][0… │
│                     │                   │            │ not_equal_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ FC1 (Dense)         │ (None, 256)       │     16,640 │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 256)       │          0 │ FC1[0][0]         │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 256)       │          0 │ activation_4[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_layer (Dense)   │ (None, 19)        │      4,883 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_5        │ (None, 19)        │          0 │ out_layer[0][0]   │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,883,563 (11.00 MB)

 Trainable params: 2,883,563 (11.00 MB)

 Non-trainable params: 0 (0.00 B)

In [102]:
model.compile(loss='binary_crossentropy',optimizer='adam',
              metrics=['accuracy'])

In [103]:
sequences_test = tok.texts_to_sequences(X_test)
sequences_matrix_test = sequence.pad_sequences(sequences_test,
                                               maxlen=max_len,padding='post')

In [104]:
model.fit(
    sequences_matrix_train, Y_train,
    batch_size=50,
    epochs=10,
    validation_data=(sequences_matrix_test, Y_test))

Epoch 1/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 29s 152ms/step - accuracy: 0.0977 - loss: 0.3746 - val_accuracy: 0.1333 - val_loss: 0.2229
Epoch 2/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 42s 156ms/step - accuracy: 0.1379 - loss: 0.2260 - val_accuracy: 0.1333 - val_loss: 0.2216
Epoch 3/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 42s 166ms/step - accuracy: 0.1431 - loss: 0.2187 - val_accuracy: 0.1333 - val_loss: 0.2206
Epoch 4/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 39s 157ms/step - accuracy: 0.1516 - loss: 0.2126 - val_accuracy: 0.1740 - val_loss: 0.1995
Epoch 5/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 42s 160ms/step - accuracy: 0.2096 - loss: 0.1892 - val_accuracy: 0.2373 - val_loss: 0.1884
Epoch 6/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 41s 161ms/step - accuracy: 0.3005 - loss: 0.1689 - val_accuracy: 0.2907 - val_loss: 0.1824
Epoch 7/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 43s 172ms/step - accuracy: 0.4015 - loss: 0.1460 - val_accuracy: 0.3547 - val_loss: 0.1678
Epoch 8/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 25s 146ms/step - accuracy: 0.5253 - loss: 0

In [94]:
from sklearn.metrics import roc_auc_score

In [105]:
predictions=model.predict(sequences_matrix_test)

47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step


In [106]:
roc_auc_score(Y_test,predictions)

np.float64(0.9045946439149674)

In [111]:
Y_test

array([[1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [114]:
pred_probs = model.predict(sequences_matrix_test)

47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step


In [115]:
threshold = 0.5  # You can tune this
pred_labels = (pred_probs >= threshold).astype(int)

In [116]:
predicted_tags = mlb.inverse_transform(pred_labels)
true_tags = mlb.inverse_transform(Y_test)

In [117]:
for i in range(5):
    print(f"\nSample {i+1}:")
    print("Text:", X_test.iloc[i][:150])  # Just showing a snippet
    print("True Tags:", true_tags[i])
    print("Predicted Tags:", predicted_tags[i])


Sample 1:
Text: impact namespaces multiple dllsi inherit vbnet project generate 2 dlls one web app another business layer sub-app larger web site use vs2005 problem s
True Tags: ('.net', 'asp.net')
Predicted Tags: ('c#',)

Sample 2:
Text: performance profile visual studio 2008 pro microsoft make piece software call visual studio 2008 professional find appear application performance prof
True Tags: ('c#',)
Predicted Tags: ('.net',)

Sample 3:
Text: edit html meta tag w aspnetquestion hello try build quick easy asp.net page redirect user new url use meta redirect trouble need also pass along get v
True Tags: ('asp.net', 'c#', 'html')
Predicted Tags: ('asp.net',)

Sample 4:
Text: differences perl python awk sedjust want know main differences among power language better use edit vs like topic information
True Tags: ('python',)
Predicted Tags: ('python',)

Sample 5:
Text: update link table ms access database c# programaticallyi two access 2003 databases four table link table two question u